**NOTA IMPORTANTE**: Las preguntas están marcadas en color rojo. Para la entrega, prepara un documento separado que solo contenga las preguntas y tus respuestas.

<font color="#f00">**La entrega es un documento PDF solo con las preguntas y respuestas**</font>

Las prácticas de este curso las haremos en Python, que tiene dos posibilidades para criptografía: el paquete `PyCryptodome` y el paquete `cryptography`. Ambos son opciones válidas e intercambiables, aunque Las prácticas de este curso las haremos con `PyCryptodome`. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [1]:
# @title
# La siguiente línea instala CryptoDome en nuestra máquina virtual
!pip install pycryptodome
# La siguiente línea nos permitirá imprimir líneas largas
from pprint import pprint
from textwrap import wrap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.9 MB/s eta 0:00:00


# Cifrado simétrico o de clave secreta

Como hemos visto en las sesiones, el cifrado simétrico o de clave secreta es aquel cifrado que utiliza la misma clave para cifrar y para descifrar. Las dos partes de la comunicación necesitan conocer la clave, y mantenerla secreta para que nadie más tenga acceso a ella.

Veremos dos ejemplos de sistemas simétricos:

- Cifrado simétrico de flujo simétrico con ChaCha20
- Cifrado simétrico de bloque con AES

## Cifrado simétrico de flujo con ChaCha20

En esta sección veremos los comandos para enviar un texto cifrado con ChaCha20.

Primero importamos las librerías que necesitamos:

In [1]:
from base64 import b64encode, b64decode
from Crypto.Cipher import ChaCha20
import Crypto.Random

ModuleNotFoundError: No module named 'Crypto'

### Configuración del sistema emisor

Los módulos de criptografía suelen necesitar una etapa inicial de configuración. Cada módulo se configura a su manera. A continuación encontrarás la etapa de configuración de ChaCha20 para PyCryptodome.

Fíjate que la clave se crea al azar con algoritmos criptográficos `Crypto.Random.get_random_bytes()`: **es fundamental que las claves sean totalmente aleatorias y creadas también con algoritmos criptográficos**. No todas las funciones de creación de azar son válidas: necesitas una fuente de azar con validez criptográfica. Muchas implementaciones de protocolos criptográficos han caído no porque el cifrado fuese erróneo, sino porque la fuente de azar no era lo suficiente "aleatoria". En particular: en criptografía, no uses la función general `random.randbytes()` sino alguna específica de la librería criptográfica, como `Crypto.Random.get_random_bytes()`. Esto es válido también para los demás sistemas criptográficos.

In [ ]:
key = Crypto.Random.get_random_bytes(32)
cipher_emisor = ChaCha20.new(key=key, nonce=None)
print(f'Longitud de la clave: {8 * len(key)} bits')

En PyCryptodome el *nonce* se puede pasar al algoritmo durante la configuración. Si, como en este caso, no se pasa *nonce* durante la creación, la librería crea un *nonce* al azar que podemos recuperar. Si decides crear tú el *nonce*, recuerda que también tiene que ser un número aleatorio creado con algoritmos criptográficos, igual que la clave.

Observa: en ChaCha20, la clave tiene 256 bits y el nonce tiene 64 bits.

In [ ]:
nonce = cipher_emisor.nonce
print(f'nonce creado automáticamente: {b64encode(nonce)}, longitud: {8 * len(nonce)} bits')

# si lo necesitas:
# nonce_creado_por_mi = Crypto.Random.get_random_bytes(8)
# cipher_emisor = ChaCha20.new(key=key, nonce=once_creado_por_mi)

### Cifrado del mensaje por el emisor

El emisor cifra el mensaje `Atacaremos al amanecer` y envía al receptor `result`, es decir, la pareja "mensaje cifrado" y "*nonce*". Fíjate: el nonce se puede enviar por un canal inseguro, así que se asume que el atacante lo conocerá.

Observa que el resultado lo codificamos en Base64 (https://es.wikipedia.org/wiki/Base64). Aunque no es necesario, sí que es común hacerlo así porque algunos protocolos (correo electrónico, JSON...) solo puede enviar caracteres imprimibles. No pierdes ni ganas seguridad si decides usar o no Base64, es más una exigencia de tu sistema de comunicaciones. Fíjate que he usado la expresión "codificamos en Base64", no ciframos. Base64 es un algoritmo de codificación de bytes, no tiene claves, cualquiera lo puede codificar y decodificar y por tanto Base64 no es un cifrado.

In [ ]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
ct = b64encode(ciphertext)
result = {'nonce':b64encode(nonce), 'ciphertext':ct}
print(result)

### Recepción y descifrado

El receptor toma el *nonce* y el *ciphertext*. Primero decodifica el base64, configura el *cipher* y el *nonce* que ha recibido y descifra. Ya veremos cómo el receptor conoce la clave, porque no se la puede enviar el emisor.

En la sesión de "Diffie-Hellman" veremos cómo el receptor conoce la clave secreta.

In [ ]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

### Siguientes mensajes: sincronización entre ciphers

Supongamos que el usuario vuelve a enviar el mismo mensaje, con el mismo cipher (fíjate que no volvemos a definir `cipher_emisor`: lo estamos reaprovechando)

In [ ]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
result = {'nonce':b64encode(nonce), 'ciphertext':b64encode(ciphertext)}
print(result)

Fíjate: estamos cifrando el mismo mensaje con el mismo nonce... pero el ciphertext es diferente. ¿Recuerdas que nunca se debe cifrar el mismo texto con la misma clave? ChaCha20 nos ayuda a que no lo hagamos, ni siquiera por equivocación, mediante el uso de un contador.

Supongamos que el receptor crea un nuevo cipher, con la misma configuración de key y nonce, e intenta descifrar:

In [ ]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

¿Qué ha pasado? ¿Por qué no se descifra? Recuerda que ChaCha20 tiene un contador adicional interno. Es decir: **emisor y receptor tienen que estar sincronizados** Es decir: para descifrar el byte número 22 tenemos que decirle al receptor que han pasado 22 bytes antes, aunque no los haya visto.

(nota: 22 es el tamaño en bytes de la cadena "Atacaremos al amanecer", que fue el contenido del primer mensaje)

Si volvemos a intentar descifrar, ahora sí que podemos hacerlo:

In [ ]:
cipher_receptor.seek(22)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

PyCryptodome y todos los demás están sincronizados siempre que descifremos los mismos bytes que hemos cifrado desde que se han creado los dos ciphers, el de emisión y el de recepción.

Si alguno de los dos pierde la sincronización (por ejemplo, porque se reinicia), entonces es necesario volver a sincronizarlos con un "seek": "ya envié XX bytes aunque no los hayas visto, mueve el estado a esta posición"

Poder volver a sincronizar los dos streams es una enorme ventaja de ChaCha20 y eso es por el parámetro `pos` autoincremental que forma parte de la matriz de estado. No todos los algoritmos permiten sincronizar los flujos si se pierde la sincronización.

## Cifrado simétrico de bloque con AES

El otro sistema de cifrado simétrico que veremos es AES. Como ya comentamos en la parte teórica, no es suficiente con indicar que usamos AES, también es necesario especificar de qué manera, es decir, **en qué modo estamos usando AES**.

Vamos a crear:

- un mensaje de 128 bits, el tamaño de bloque de AES.
- una clave k de 128 bits que usaremos durante todo el ejercicio.

In [12]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

m = b'Atacad a las dos'
k = get_random_bytes(16)
print(f'Mensaje: "{m}" Tamaño={len(m) * 8} bits')
print(f'Clave: {b64encode(k)} Tamaño={len(k) * 8} bits')

Mensaje: "b'Atacad a las dos'" Tamaño=128 bits
Clave: b'0z5Yb3/1/IuqAV1AmJGE2A==' Tamaño=128 bits


### Modo ECB

El primer modo que veremos es "ECB": cada bloque se cifra de forma independiente, sin realimentación con los demás.

![center w:30em](https://github.com/Juanvvc/crypto/blob/master/ejercicios/03/images/ECB_encryption.svg?raw=1)

Vamos a ciframos tres veces el mismo mensaje. Observa que no hay memoria, y que cifrar dos veces el mismo mensaje con la misma clave produce el mismo texto cifrado. Por eso no se debe usar nunca el modo ECB: un atacante que observa las comunicaciones quizá no sepa qué estamos diciendo, pero sí que sabe que estamos repitiendo el mensaje y eso puede ser suficiente para sus objetivos.

**Nunca debe usarse AES en modo ECB**

In [13]:
cipher = AES.new(k, AES.MODE_ECB)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
c3 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))
print(b64encode(c3))

b'V5hraNZukPNB6RUwrOrPYA=='
b'V5hraNZukPNB6RUwrOrPYA=='
b'V5hraNZukPNB6RUwrOrPYA=='


In [ ]:
decipher = AES.new(k, AES.MODE_ECB)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
m3 = decipher.decrypt(c3)
print(m1)
print(m2)
print(m3)

### Modo CBC

En el modo CBC, hay realimentación entre bloques y existe un vector de inicialización

![](https://github.com/Juanvvc/crypto/blob/master/ejercicios/03/images/CBC_encryption.svg?raw=1)

Ciframos dos veces el mismo mensaje. Observa que tenemos que crear un IV (Vector de Inicialización), y que este IV se lo tenemos que enviar al receptor. El envío del IV se hace durante el primer mensaje, antes de que el canal sea seguro, pero no hay problemas en que un atacante conozca el IV.

In [14]:
iv = get_random_bytes(16)
cipher = AES.new(k, AES.MODE_CBC, iv=iv)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))

b'ZDcU1YpqUO00S07mVniRmg=='
b'+RBipbf4TIPblbpdkwST2g=='


<font color="red">

<b>PREGUNTA:</b>

1. Ahora los dos cifrados son diferentes a pesar de que estamos cifrando el mismo mensaje. ¿Por qué sucede eso?
</font>

Descifrado: necesita la clave y el IV. La clave es secreta y el receptor tiene que haberla recibido por un canal secreto (lo veremos) pero el IV puede recibirse sin protección al inicio de la comunicación.

In [15]:
decipher = AES.new(k, AES.MODE_CBC, iv=iv)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
print(m1)
print(m2)

b'Atacad a las dos'
b'Atacad a las dos'


### Ejercicio **opcional**

- ¿Puedes programar el modo CBC a partir del modo ECB? ECB es la caja AES básica, así que es posible programar (¡como ejercicio solamente!) el modo CBC como composición de ECB
- ¿Puedes programar los demás modos?

Ejemplo de solución (solo parte de cifrado) de la primera pregunta. Observa que el resultado es el mismo de antes al cifrar m en modo CBC

In [ ]:
from Crypto.Util.strxor import strxor

class AES_CBC():
    def __init__(self, k, iv):
        self.iv = iv
        self.cipher = AES.new(k, AES.MODE_ECB)
    def encrypt(self, msg):
        # primero hacemos XOR del mensaje con el IV que tenemos
        m = strxor(msg, self.iv)
        c = self.cipher.encrypt(m)
        # para la siguiente ronda, el IV es el propio texto cifrado
        self.iv = c
        return c

k = get_random_bytes(16)
iv = get_random_bytes(16)
mycbc = AES_CBC(k, iv)
print(b64encode(mycbc.encrypt(m)))
print(b64encode(mycbc.encrypt(m)))

### Padding

¿Qué pasa si tenemos que enviar mensajes más cortos que la longitud de bloque de AES? Entonces tenemos que usar algún algoritmo de padding. Es decir: marcar la longitud del mensaje original. Observa que solo podemos enviar bloques de 128 bits, y si intentamos enviar bloques más cortos o más largos saltará un error:

In [16]:
c1 = cipher.encrypt(b'mensaje corto')

ValueError: Data must be padded to 16 byte boundary in CBC mode

Con Cryptodome podemos usar las funciones pad() y unpad()

Observa: en este ejemplo no especificamos IV al configurar el cipher, pero ya sabes que en modo CBC siempre hay un IV. Si no lo especificamos, escogerá el IV al azar utilizando algoritmos segutos, y eso es muy adecuado. Tenemos que enviarle ese IV al receptor en el primer mensaje.


In [18]:
from Crypto.Util.Padding import pad, unpad

# mensaje corto
m = b'1234'
cipher = AES.new(k, AES.MODE_CBC)
c = cipher.encrypt(pad(m, AES.block_size))
print({'iv':b64encode(cipher.iv), 'ciphertext':b64encode(c)})

{'iv': b'FHAKL+vnlweGw2J334KYXA==', 'ciphertext': b'y0PMup02DMyq3nKJqS04kA=='}


Recepción:

In [19]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = unpad(decipher.decrypt(c), AES.block_size)
print(f"The message was: {pt}")

The message was: b'1234'


¿Qué pasa si no usamos unpad? AES es un cifrado de bloque, así que los mensajes en AES tienen obligatoriamente un tamaño igual al bloque AES (128 bits), así que vemos el *padding* que sobra. Las función *unpad()* nos hubiese cortado esos bytes sobrantes.

In [20]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = decipher.decrypt(c)
print(f"The message was: {pt} (longitud {len(pt) * 8} bits)")

The message was: b'1234\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c' (longitud 128 bits)


<font color="red">
<b>PREGUNTA:</b>

1. Vimos que ChaCha20 podía resincronizarse si se perdían mensajes. ¿AES en modo CBC puede resincronizarse si se pierden mensajes? ¿Qué modo de AES permitiría resincronizarse?

</font>